In [1]:
from prophet import Prophet, utilities
import pandas as pd
import numpy as np
import random

In [2]:
df_feat = pd.read_csv('../examples/regressor_constraints_example.csv')
df_feat

,ds,series_index,y,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6
0,2017-10-03,2,229.680,0.108489,0,0,0,0,0,0
1,2017-10-04,2,316.656,0.105098,0,0,0,0,0,0
2,2017-10-05,2,144.326,0.126457,0,0,0,0,0,0
3,2017-10-06,2,361.782,0.104484,0,0,0,0,0,0
4,2017-10-07,2,128.100,0.107427,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
128688,2022-01-27,417,NaN,0.200000,0,0,0,0,0,0
128689,2022-01-28,417,NaN,0.200000,0,0,0,0,0,0
128690,2022-01-29,417,NaN,0.200000,0,0,0,0,0,0
128691,2022-01-30,417,NaN,0.200000,0,0,0,0,0,0


In [3]:
#These are the possible values to constraint parametr
#Positive = coef can only be positive
#Negative = coef can only be negative
#Unconstrained (default) = coef can be any value

possible_constraint = ["positive", "negative", "unconstrained"]

total_coef = pd.DataFrame()
total_fcst = pd.DataFrame()

for index, group in df_feat.groupby('series_index'):
    
    m = Prophet(growth='linear', daily_seasonality=False, changepoint_range=0.9)
    feat_cols = [col for col in df_feat.columns if 'feature' in col]
    
    for feature in feat_cols:
        #You must pass the parameter inside the add_regressor method
        m.add_regressor(feature, constraint=random.choice(possible_constraint))
        group[feature] = group[feature].ffill()
            
    m.fit(group.dropna(subset=['y']))
    fcst = m.predict(group)
    
    # Getting constraint by regressor
    for feat,params in m.extra_regressors.items():
        fcst[f'{feat}_constraint'] = params['constraint']
    
    # Prophet utilities to get regressor coefficients
    coefs = utilities.regressor_coefficients(m)
    coefs['col'] = coefs['regressor'] + '_' + coefs['regressor_mode']
    coefs = coefs.rename(columns={'center':'value_center'}).drop(columns=['regressor','regressor_mode'])
    coefs = coefs.set_index(['col']).unstack(['col']).to_frame().T.reorder_levels([1, 0],axis=1)
    coefs.columns = ['_'.join(col) for col in coefs.columns]
    coefs = coefs.assign(**{'key':1})
    fcst = fcst.assign(**{'key':1}).merge(coefs, on='key').drop(columns='key')
    fcst['series_index'] = index 
    
    total_fcst = pd.concat([total_fcst, fcst], sort=True)
    print(f'finished {index}')

finished 2
finished 3
finished 7
finished 11
finished 16
finished 18
finished 21
finished 35
finished 40
finished 44
finished 49
finished 53
finished 65
finished 67
finished 72
finished 75
finished 82
finished 86
finished 101
finished 105
finished 107
finished 108
finished 112
finished 113
finished 121
finished 136
finished 150
finished 158
finished 159
finished 160
finished 171
finished 179
finished 183
finished 190
finished 191
finished 192
finished 196
finished 201
finished 208
finished 220
finished 230
finished 231
finished 237
finished 239
finished 247
finished 249
finished 251
finished 253
finished 256
finished 268
finished 270
finished 272
finished 274
finished 275
finished 277
finished 284
finished 292
finished 302
finished 304
finished 307
finished 314
finished 315
finished 320
finished 321
finished 331
finished 341
finished 346
finished 348
finished 349
finished 356
finished 367
finished 370
finished 371
finished 374
finished 378
finished 381
finished 382
finished 385
finishe

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


finished 404


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


finished 410


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


finished 414
finished 417


In [4]:
#Treating dataframe to get constraints and coefs

feat_cols = [col for col in df_feat.columns if 'feature' in col]
eval_results = total_fcst[['series_index']+[f'{feat}_{col}' for feat in feat_cols for col in ['additive_coef','constraint']]].drop_duplicates()
eval_results = eval_results.melt(['series_index'],var_name='feature',value_name='coef')
eval_results['param'] = np.where(eval_results['feature'].str.contains('coef'),'coef','constraint')
eval_results['col'] = eval_results['feature'].str.split('_').str.slice(stop=2).str.join(sep='_')
eval_results = eval_results.drop(columns=['feature'])
eval_results = eval_results.set_index(['series_index','col','param']).unstack('param').droplevel(0,axis=1).reset_index()

# Criteria to fail 
# If constraint == 'negative' and coef > 0
# If constraint == 'positive' and coef < 0

eval_results['fail'] = np.where((eval_results['constraint']=='negative') & (eval_results['coef'] > 0), 1,
                               np.where((eval_results['constraint']=='positive') & (eval_results['coef'] < 0), 1, 0))
eval_results

param,series_index,col,coef,constraint,fail
0,2,feature_0,-407.996,unconstrained,0
1,2,feature_1,0,negative,0
2,2,feature_2,0,unconstrained,0
3,2,feature_3,0,unconstrained,0
4,2,feature_4,1.19964,positive,0
...,...,...,...,...,...
597,417,feature_2,0.227102,unconstrained,0
598,417,feature_3,0.141249,positive,0
599,417,feature_4,0.106086,positive,0
600,417,feature_5,0.0707045,unconstrained,0


In [5]:
#If sum equals zero, no fails have happened
eval_results['fail'].sum()

0